<a href="https://colab.research.google.com/github/dudasimonassi/HackathonIA/blob/main/Competicao_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Competição de IA: Desafio de Previsão de Preços de Imóveis

Etapas para compilação correta do código:
1.   Realizar o download da base de dados e colocar no Drive;
2.   Mudar o caminho no código para a leitura do arquivo.

Dicas e boas estratégias para se ter uma boa previsão:
1.   Implementar outras etapas de pré-processamento (outliers, normalização, etc);
2.   Observar o comportamento dos dados;
3.   Utilizar outros modelos de regressão;
4.   Usar um otimizador de hyperparâmetros;
5.   Analizar as diferenças entre os valores reais e previstos;


**Fazer a submissão no formulário antes do prazo de encerramento.**

Links:
*   Regulamento: https://drive.google.com/file/d/1MA2KaBSQTOh0HMG8yMneW2Qdw0wlMxxC/view?usp=share_link
*   Base de Dados: https://docs.google.com/spreadsheets/d/140EhqT7kTrRhys20-bBeA2nXZtx2JVki/edit?usp=share_link&ouid=103873027701310997143&rtpof=true&sd=true
*   Formulário: https://forms.gle/ASLn711xNrZtzfDv7



## Bibliotecas

**OBS:** Essas são bibliotecas recomendadas, não sendo obrigatória a sua utilização. O uso de bibliotecas adicionais é permitido.

In [ ]:
# Processamento de matrizes e funções matemáticas
import numpy as np
# Manipulação e análise de dados
import pandas as pd
# Plotagem de gráficos
import matplotlib.pyplot as plt
# Distribuição de dados para o treino e teste
from sklearn.model_selection import train_test_split
# Métricas
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error

# Funções matemáticas
import math
import random

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
# Modelos (apenas como ilustração)
from sklearn.linear_model import LinearRegression

In [ ]:
# Localização para o Teste Online
from geopy.geocoders import Nominatim

## Dados

**OBS:** Apenas são permitidas modificações na base de dados utilizando linhas de código.

### Importação da Base de dados

In [ ]:
# Executando a célula para montar o Drive
from google.colab import drive
drive.mount('/content/drive')

**MUDAR O CAMINHO DO ARQUIVO**

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Competicao_IA_Dados.xlsx')
df

### Tratamento

**OBS:** Essas são etapas de facilitação para a realização dos códigos

In [ ]:
# Corrigindo os nomes para 'Galpão' e 'Salão' corretamente
df['Tipo'] = df['Tipo'].replace({'Ã£': 'ã'}, regex=True)
# Corrigindo o nome para 'Prédio' corretamente
df['Tipo'] = df['Tipo'].replace({'Ã©': 'é'}, regex=True)
# Corrigindo o nome para 'Área' corretamente
df['Tipo'] = df['Tipo'].replace({'Ã_x0081_': 'Á'}, regex=True)
# Retirando as colunas que não são usadas
df = df.drop(['ID', 'Cidade', 'Bairro'], axis= 1)
# Retirando os Tipos de Imóveis não utilizados
df = df.drop(df[df['Tipo'].isin(['Salão', 'Terreno', 'Prédio', 'Sobrado', 'Flat', 'Ponto', 'Área', 'Laje'])].index)
# Retirando os dados desnecessários
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

# Transformando os dados de preços em 'float'
money_col = df[['Preço', 'Condomínio', 'IPTU']]
for col in money_col:
    df[col] = df[col].astype(str).apply(lambda x: x.replace('.',''))
    df[col] = df[col].replace({'R\$ ': '', ',': '.'}, regex=True).astype('float64')

# Transformando os dados 'float' em 'int'
int_col = df[['Quartos', 'Banheiros', 'Vagas']]
for col in int_col:
    df[col] = df[col].astype(int)
df = df.reset_index()
df = df.drop(['index'], axis= 1)

In [ ]:
lista = df.duplicated()

for index, row in df.iterrows():
    if(lista[index] == True):
      print(row)

In [ ]:
#Remove linhas com zero na coluna m2
df = df[df['m2'] != 0]

#Normaliza longitude e latitude
scaler = MinMaxScaler(feature_range=(0, 1))
df['Longitude'] = scaler.fit_transform(df[['Longitude']])
df['Latitude'] = scaler.fit_transform(df[['Latitude']])

#Separa os dataframes de acordo com o tipo de locação
apartamento_cobertura_df = df[((df['Tipo'] == 'Apartamento') | (df['Tipo'] == 'Cobertura')) & ((df['Quartos'] != 0) & (df['Banheiros'] != 0))]
studio_kitnet_df = df[((df['Tipo'] == 'Studio') | (df['Tipo'] == 'Kitnet')) & (df['Quartos'] <= 1) & (df['Banheiros'] != 0)]
studio_kitnet_mais_de_um_quarto_df = df[((df['Tipo'] == 'Studio') | (df['Tipo'] == 'Kitnet')) & ((df['Quartos'] > 1 & ((df['Banheiros'] != 0))))]
galpao_df = df[df['Tipo'] == 'Galpão']
casa_df = df[(df['Tipo'] == 'Casa')  & ((df['Quartos'] != 0) & (df['Banheiros'] != 0))]
loja_df = df[df['Tipo'] == 'Loja']
sala_df = df[df['Tipo'] == 'Sala']
#cobertura_df = df[df['Tipo'] == 'Cobertura']
#kitnet_df = df[df['Tipo'] == 'Kitnet']

Analise de semelhança entre studios e kitnets com mais de um quarto e apartamentos

In [ ]:
apartamento_cobertura_df.describe()

In [ ]:
studio_kitnet_df.describe()

In [ ]:
studio_kitnet_mais_de_um_quarto_df.describe()

In [ ]:
#Junção de apartamentos com kinet e studios com mais de um quarto -> semelhança encontrada
apartamento_cobertura_kitnet_df = pd.concat([apartamento_cobertura_df, studio_kitnet_mais_de_um_quarto_df])

In [ ]:
#Plotagem para comparação
apartamento_cobertura_df['Preço'].plot(kind='hist', title="Apartamento e Cobertura")
plt.show()
apartamento_cobertura_kitnet_df['Preço'].plot(kind='hist', title="Apartamento, Cobertura e Kitnet")
plt.show()
studio_kitnet_df['Preço'].plot(kind='hist', title="Studio e Kitnet")
plt.show()
galpao_df['Preço'].plot(kind='hist', title="Galpão")
plt.show()
casa_df['Preço'].plot(kind='hist', title="Casa")
plt.show()
loja_df['Preço'].plot(kind='hist', title="Loja")
plt.show()
sala_df['Preço'].plot(kind='hist', title="Sala")

In [ ]:
#Plotagem para comparação
apartamento_cobertura_df.boxplot(column=['Preço'])
plt.show()
apartamento_cobertura_kitnet_df.boxplot(column=['Preço'])
plt.show()
studio_kitnet_df.boxplot(column=['Preço'])
plt.show()
galpao_df.boxplot(column=['Preço'])
plt.show()
casa_df.boxplot(column=['Preço'])
plt.show()
loja_df.boxplot(column=['Preço'])
plt.show()
sala_df.boxplot(column=['Preço'])
plt.show()

In [ ]:
#Retira os outliers
"""apartamento_cobertura_df.boxplot(column=['Preço'])
plt.show()
print(apartamento_cobertura_df.shape)
Q1 = apartamento_cobertura_df['Preço'].quantile(0.25)
Q3 = apartamento_cobertura_df['Preço'].quantile(0.75)
IQR = Q3 - Q1

mask = ((apartamento_cobertura_df['Preço'] < (Q1 - 1.5 * IQR)) | (apartamento_cobertura_df['Preço'] > (Q3 + 1.5 * IQR)))
apartamento_cobertura_df[mask] = np.nan

apartamento_cobertura_df = apartamento_cobertura_df.dropna()

apartamento_cobertura_df.boxplot(column=['Preço'])
plt.show()
print(apartamento_cobertura_df.shape)"""

Tipos de Imóveis a serem avaliados:
*   Apartamento
*   Casa
*   Studio
*   Loja
*   Sala
*   Galpão

**OBS:** Ainda existem dois tipos de imóveis, Cobertura (Apartamento) e Kitnet (Studio). O que será feito com eles, cabe a você decidir.
*   Cobertura é um tipo de apartamento;
*   Kitnet é um termo que não é mais utilizado, se assemelha a um Studio.





## Modelo

**OBS:** Todos os modelos de regressão são permitidos.


**Algumas observações**
*   '*test_size*': Distribuição em porcentagem para dados de teste;
*   '*random_state*': Forma que os dados são separados (aleatoriedade).

In [ ]:
# NOSSO
# Inputs
X = apartamento_cobertura_kitnet_df.drop(['Preço'], axis=1)
# Outputs
y = [i[0] for i in apartamento_cobertura_kitnet_df[['Preço']].values]
# Dados de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

# Remoção do Tipo de Imóvel para o código compilar (para uma boa prevista, recomenda-se utilizar esse atributo de alguma forma)
X_train = X_train.drop(['Tipo'], axis=1)
X_test = X_test.drop(['Tipo'], axis=1)

# Modelo KNN
model = KNeighborsRegressor(n_neighbors=2)
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

In [ ]:
def runKNN(dataframe, random_state):
  X = dataframe.drop(['Preço'], axis=1)
  # Outputs
  y = [i[0] for i in dataframe[['Preço']].values]
  # Dados de treino e teste
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)

  # Remoção do Tipo de Imóvel para o código compilar (para uma boa prevista, recomenda-se utilizar esse atributo de alguma forma)
  X_train = X_train.drop(['Tipo'], axis=1)
  X_test = X_test.drop(['Tipo'], axis=1)

  # Modelo KNN
  model = KNeighborsRegressor(n_neighbors=2)
  model.fit(X_train, y_train)
  y_predict = model.predict(X_test)

  R2 = r2_score(y_test, y_predict)
  print("R2   =", R2)
  RMSE = math.sqrt(mean_squared_error(y_test, y_predict))
  print("RMSE =", RMSE)
  MAPE = mean_absolute_percentage_error(y_test, y_predict)
  print("MAPE =", MAPE)

#runKNN(apartamento_cobertura_kitnet_df, 16)
#runKNN(studio_kitnet_df)
#runKNN(galpao_df)
#runKNN(casa_df)
#runKNN(loja_df)
#runKNN(sala_df)

In [ ]:
def runRandomForest(dataframe):
  X = dataframe.drop(['Preço'], axis=1)
  # Outputs
  y = [i[0] for i in dataframe[['Preço']].values]
  # Dados de treino e teste
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

  # Remoção do Tipo de Imóvel para o código compilar (para uma boa prevista, recomenda-se utilizar esse atributo de alguma forma)
  X_train = X_train.drop(['Tipo'], axis=1)
  X_test = X_test.drop(['Tipo'], axis=1)

  # Modelo Random Forest
  model = RandomForestClassifier()
  model.fit(X_train, y_train)
  y_predict = model.predict(X_test)

  R2 = r2_score(y_test, y_predict)
  print("R2   =", R2)
  RMSE = math.sqrt(mean_squared_error(y_test, y_predict))
  print("RMSE =", RMSE)
  MAPE = mean_absolute_percentage_error(y_test, y_predict)
  print("MAPE =", MAPE)


#runRandomForest(apartamento_cobertura_kitnet_df)
#runRandomForest(studio_kitnet_df)
#runRandomForest(galpao_df)
#runRandomForest(casa_df)
#runRandomForest(loja_df)
#runRandomForest(sala_df)

In [ ]:
"""# Definindo o modelo -> Regressão Linear Padrão
model = LinearRegression(fit_intercept=True, copy_X=True, n_jobs=None, positive=False)
# Treinando o modelo
model.fit(X_train, y_train)
# Prevendo os resultados
y_predict = model.predict(X_test)"""

"""# Modelo Random Forest
model = RandomForestClassifier()
model.fit(X_train, y_train)
y_predict = model.predict(X_test)"""

### Aplicação

**OBS:** Modelos de regressão não conseguem prever textos (ex: Apartamento 'R$'), o que você irá fazer com esse atributo?

## Métricas

São obrigatórias a avaliação das seguintes métricas:



*   $R^2$: Quanto mais próximo de 1, melhor o resultado
*   RMSE: Quanto menor o valor, melhor o resultado
*   MAPE: Quanto mais próximo de 0, melhor o resultado

**OBS:** Mais métricas para análise podem ser utilizadas.

In [ ]:
R2 = r2_score(y_test, y_predict)
print("R2   =", R2)
RMSE = math.sqrt(mean_squared_error(y_test, y_predict))
print("RMSE =", RMSE)
MAPE = mean_absolute_percentage_error(y_test, y_predict)
print("MAPE =", MAPE)

R2   = 0.7947594908857658
RMSE = 673.0399292973423
MAPE = 0.2249444193595549


## Teste Online

**OBS:** Pode mudar essa parte de acordo com as suas necessidades.

In [ ]:
# Imputando os dados
ty = input("Tipo do Imóvel: ")
m_2 = int(input("Metros quadrados: "))
n_q = int(input("Número de quartos: "))
n_b = int(input("Número de banheiros: "))
n_vg = int(input("Número de vagas de garagem: "))
cond = float(input("Preço do Condomínio: "))
iptu = float(input("Preço do IPTU: "))
bairro = input("Bairro (texto): ")
# Transformando o bairro em texto para Lat, Lon
geolocator = Nominatim(user_agent="geolocalização")
location = geolocator.geocode(bairro + ', Juiz de Fora - MG')
lat = location.latitude; lon = location.longitude

# Colocando os dados em um Dataframe
col_online = ['m2', 'Quartos', 'Banheiros', 'Vagas', 'Condomínio', 'IPTU',	'Latitude', 'Longitude']
X_online = [m_2, n_q, n_b, n_vg, cond, iptu, lat, lon]
X_online = pd.DataFrame([X_online], columns = col_online)

# Prevendo o valor do preço do imóvel
y_online = model.predict(X_online) #OBS: Colocar o nome do seu modelo treinado

# Resultado
print(f"Preço do imóvel previsto:{y_online}")